In [2]:
from urllib.request import urlopen
import timm
import torch
import zipfile,os
from PIL import Image
from pathlib import Path
from torchvision.datasets import ImageFolder
from torchvision import transforms
from torch.utils.data import DataLoader,Dataset
from sklearn.metrics import accuracy_score

c:\Users\bryan\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
torch.cuda.is_available()

False

In [4]:
local_zip = '/DATA/ai lab/gcdData/GCD.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/DATA/ai lab')
zip_ref.close()

In [5]:
train_dir = '/DATA/ai lab/GCD/train'
test_dir = '/DATA/ai lab/GCD/test'

os.listdir('/DATA/ai lab/GCD/train')
print(train_dir)

/DATA/ai lab/GCD/train


In [ ]:
model = timm.create_model(
    'maxvit_tiny_tf_224.in1k',
    pretrained=True,
    num_classes=7,
)
model = model.eval()

# get model specific transforms (normalization, resize)
data_config = timm.data.resolve_model_data_config(model)
trans = timm.data.create_transform(**data_config, is_training=False)

In [7]:
'''
train_dir = Path(train_dir)

for img_path in train_dir.rglob("*.*"):  # Recursively find all files
    if img_path.suffix.lower() in [".jpg", ".jpeg", ".png"]:  # Check for valid image extensions
        img = Image.open(img_path).convert("RGB")  # Open the image and convert to RGB
        img_tensor = transforms(img).unsqueeze(0)  # Preprocess and add batch dimension

        # Pass the image through the model
        output = model.forward_features(transforms(img_tensor))

        print(f"Processed {img_path}: Output Shape - {output[-1].shape if isinstance(output, list) else output.shape}")
'''

'\ntrain_dir = Path(train_dir)\n\nfor img_path in train_dir.rglob("*.*"):  # Recursively find all files\n    if img_path.suffix.lower() in [".jpg", ".jpeg", ".png"]:  # Check for valid image extensions\n        img = Image.open(img_path).convert("RGB")  # Open the image and convert to RGB\n        img_tensor = transforms(img).unsqueeze(0)  # Preprocess and add batch dimension\n\n        # Pass the image through the model\n        output = model.forward_features(transforms(img_tensor))\n\n        print(f"Processed {img_path}: Output Shape - {output[-1].shape if isinstance(output, list) else output.shape}")\n'

In [13]:
class CustomDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.data_dir = data_dir
        self.transform = transform
        self.images = []
        self.labels = []
        self.class_names = os.listdir(data_dir)

        for label, class_name in enumerate(self.class_names):
            class_dir = os.path.join(data_dir, class_name)
            for img_name in os.listdir(class_dir):
                img_path = os.path.join(class_dir, img_name)
                self.images.append(img_path)
                self.labels.append(label)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = self.images[idx]
        image = Image.open(img_path).convert('RGB')
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label  # Ensure this returns a tuple of (image, label)

transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224
    transforms.ToTensor(),           # Convert images to PyTorch tensors
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize
])

# Create an instance of the CustomDataset
dataset = CustomDataset(data_dir=train_dir, transform=transform)

train_loader = DataLoader(dataset, batch_size=32, shuffle=True)
for data in train_loader:
    print(data[0].shape)  # This will show you the structure of the data being returned
    inputs, targets = data  # Unpack only if it has the correct structure
    break

AttributeError: 'Compose' object has no attribute 'Compose'

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_epochs = 20
# Training loop
model.train()  # Set the model to training mode
for epoch in range(num_epochs):
    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device)  # Move to device

        optimizer.zero_grad()  # Zero the gradients
        outputs = model(inputs)  # Forward pass
        loss = criterion(outputs, targets)  # Compute loss
        loss.backward()  # Backward pass
        optimizer.step()  # Update weights

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

# After training, you can evaluate the model as you did before.

NameError: name 'model' is not defined

In [ ]:
test_dataset = ImageFolder(root=test_dir, transform=transforms)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# 4. Define testing loop
correct_predictions = 0
total_samples = 0
all_targets = []
all_preds = []

'''
with torch.no_grad():  # Disable gradient computation for inference
    for inputs, targets in test_loader:
        # Perform forward pass
        outputs = model(inputs)
        
        # Get predicted class indices
        _, preds = torch.max(outputs, dim=1)
        
        # Compare predictions with ground truth
        correct_predictions += (preds == targets).sum().item()
        total_samples += targets.size(0)
        
        # Store predictions and labels for metrics
        all_targets.extend(targets.tolist())
        all_preds.extend(preds.tolist())
'''
# Perform forward pass
outputs = model(inputs)
        
# Get predicted class indices
_, preds = torch.max(outputs, dim=1)
        
# Compare predictions with ground truth
correct_predictions += (preds == targets).sum().item()
total_samples += targets.size(0)
        
# Store predictions and labels for metrics
all_targets.extend(targets.tolist())
all_preds.extend(preds.tolist())        

accuracy = accuracy_score(all_targets, all_preds)
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 0.00%
